In [71]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil

In [72]:
from szp_funcs import load_month, re_szp, re_szp_job, load_groups, print_df, months_year, custom_create_res, re_fot, re_fot_job, load_month_sum_by_job, accum_fot_job

In [73]:
months = months_year[:5]
path_to_dir = 'C:\\Users\\PetukhovMD\\Documents\\code\\montly reports\\counting\\Accum + q stats\\'

In [74]:
months_year_rus = ['янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']

In [75]:
res = custom_create_res(months, load_month)

In [76]:
for_fot = custom_create_res(months, load_month_sum_by_job, on=['inn', 'snils', 'type', 'job'])

In [77]:
ped, isp, aup = load_groups('input')

In [78]:
def give_grp_from_matcher(row, matcher, name_of_return='szp'):
    if row['inn'] in matcher:
        if row['snils'] in matcher[row['inn']]:
            return row[name_of_return]
    return np.nan

In [79]:
for i in range(len(months)):
    t = i + 1
    if t % 3 != 0:
        continue
    tm = months[:t]
    res['szp_q_' + str(int(t/3))] = res.apply(lambda row: re_szp(row, tm), axis=1)
    res['szp_teach_q_' + str(int(t/3))] = res.apply(lambda row: re_szp_job(row, tm, ['Учитель']), axis=1)
    res['szp_ped_q_' + str(int(t/3))] = res.apply(lambda row: re_szp_job(row, tm, ped), axis=1)
    res['fot_q_' + str(int(t/3))] = res.apply(lambda row: re_fot(row, tm), axis=1)
    res['fot_aup_q_' + str(int(t/3))] = res.apply(lambda row: re_fot_job(row, tm, aup), axis=1)
    df_all = res[res['szp_q_' + str(int(t/3))].isna() == False][['inn', 'snils', 'szp_q_' + str(int(t/3))]]
    df_teach = res[res['szp_teach_q_' + str(int(t/3))].isna() == False][['inn', 'snils', 'szp_teach_q_' + str(int(t/3))]]
    df_ped = res[res['szp_ped_q_' + str(int(t/3))].isna() == False][['inn', 'snils', 'szp_ped_q_' + str(int(t/3))]]
    df_all = df_all.groupby('inn')
    df_ped = df_ped.groupby('inn')
    df_teach = df_teach.groupby('inn')
    top10_all = {}
    bot10_all = {}
    top10_teach = {}
    bot10_teach = {}
    bot50_teach = {}
    bot50_ped = {}
    for group in df_all.groups:
        cnt = df_all.get_group(group)['snils'].nunique()
        top10_all[group] = df_all.get_group(group).sort_values(by='szp_q_' + str(int(t/3)), ascending=False, axis=0).head(ceil(cnt / 10))['snils'].to_list()
        bot10_all[group] = df_all.get_group(group).sort_values(by='szp_q_' + str(int(t/3)), ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()
    for group in df_teach.groups:
        cnt = df_teach.get_group(group)['snils'].nunique()
        top10_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach_q_' + str(int(t/3)), ascending=False, axis=0).head(ceil(cnt / 10))['snils'].to_list()
        bot10_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach_q_' + str(int(t/3)), ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()
        bot50_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach_q_' + str(int(t/3)), ascending=False, axis=0).tail(ceil(cnt / 2))['snils'].to_list()
    for group in df_ped.groups:
        cnt = df_ped.get_group(group)['snils'].nunique()
        bot50_ped[group] = df_ped.get_group(group).sort_values(by='szp_ped_q_' + str(int(t/3)), ascending=False, axis=0).tail(ceil(cnt / 2))['snils'].to_list()
    res['top10_all_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, top10_all, 'szp_q_' + str(int(t/3))), axis=1)
    res['bot10_all_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, bot10_all, 'szp_q_' + str(int(t/3))), axis=1)
    res['top10_teach_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, top10_teach, 'szp_teach_q_' + str(int(t/3))), axis=1)
    res['bot10_teach_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, bot10_teach, 'szp_teach_q_' + str(int(t/3))), axis=1)
    res['bot50_teach_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, bot50_teach, 'szp_teach_q_' + str(int(t/3))), axis=1)
    res['bot50_ped_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, bot50_ped, 'szp_ped_q_' + str(int(t/3))), axis=1)

In [80]:
res['szp'] = res.apply(lambda row: re_szp(row, months), axis=1)
res['szp_teach'] = res.apply(lambda row: re_szp_job(row, months, ['Учитель']), axis=1)
res['szp_ped'] = res.apply(lambda row: re_szp_job(row, months, ped), axis=1)
res['fot'] = res.apply(lambda row: re_fot(row, months), axis=1)
res['fot_aup'] = res.apply(lambda row: re_fot_job(row, months, aup), axis=1)

In [81]:
df_all = res[res.szp.isna() == False][['inn', 'snils', 'szp']]
df_teach = res[res.szp_teach.isna() == False][['inn', 'snils', 'szp_teach']]
df_ped = res[res.szp_ped.isna() == False][['inn', 'snils', 'szp_ped']]

In [82]:
df_all = df_all.groupby('inn')
df_ped = df_ped.groupby('inn')
df_teach = df_teach.groupby('inn')

In [83]:
top10_all = {}
bot10_all = {}
top10_teach = {}
bot10_teach = {}
bot50_teach = {}
bot50_ped = {}

In [85]:
for group in df_all.groups:
    cnt = df_all.get_group(group)['snils'].nunique()
    top10_all[group] = df_all.get_group(group).sort_values(by='szp', ascending=False, axis=0).head(ceil(cnt / 10))['snils'].to_list()
    bot10_all[group] = df_all.get_group(group).sort_values(by='szp', ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()

In [86]:
for group in df_teach.groups:
    cnt = df_teach.get_group(group)['snils'].nunique()
    top10_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach', ascending=False, axis=0).head(ceil(cnt / 10))['snils'].to_list()
    bot10_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach', ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()
    bot50_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach', ascending=False, axis=0).tail(ceil(cnt / 2))['snils'].to_list()

In [87]:
for group in df_ped.groups:
    cnt = df_ped.get_group(group)['snils'].nunique()
    bot50_ped[group] = df_ped.get_group(group).sort_values(by='szp_ped', ascending=False, axis=0).tail(ceil(cnt / 2))['snils'].to_list()

In [88]:
res['top10_all'] = res.apply(lambda row: give_grp_from_matcher(row, top10_all), axis=1)
res['bot10_all'] = res.apply(lambda row: give_grp_from_matcher(row, bot10_all), axis=1)
res['top10_teach'] = res.apply(lambda row: give_grp_from_matcher(row, top10_teach, 'szp_teach'), axis=1)
res['bot10_teach'] = res.apply(lambda row: give_grp_from_matcher(row, bot10_teach, 'szp_teach'), axis=1)
res['bot50_teach'] = res.apply(lambda row: give_grp_from_matcher(row, bot50_teach, 'szp_teach'), axis=1)
res['bot50_ped'] = res.apply(lambda row: give_grp_from_matcher(row, bot50_ped, 'szp_ped'), axis=1)

In [90]:
for_fot['fot'] = for_fot.apply(lambda row: re_fot(row, months), axis=1)
for_fot['fot_aup'] = for_fot.apply(lambda row: accum_fot_job(row, months, aup), axis=1)
for_fot['fot_isp'] = for_fot.apply(lambda row: accum_fot_job(row, months, isp), axis=1)

In [91]:
cnts = res.groupby('inn')[['top10_all', 'top10_teach', 'bot50_teach', 'bot50_ped']].count().reset_index().rename(columns={'top10_all': 'dezil_cnt', 'top10_teach': 'dezil_teach_cnt', 'bot50_teach': 'teach_diff_cnt', 'bot50_ped': 'ped_diff_cnt'})
means = res.groupby('inn')[['szp', 'szp_ped', 'szp_teach', 'top10_all', 'bot10_all', 'top10_teach', 'bot10_teach', 'bot50_teach', 'bot50_ped']].mean().reset_index()
stats = pd.merge(means, cnts)
stats['dezil'] = stats.top10_all / stats.bot10_all
stats['dezil_teach'] = stats.top10_teach / stats.bot10_teach
stats['teach_diff'] = stats.szp_teach / stats.bot50_teach
print_df(pd.merge(stats, for_fot[for_fot.type != 'Внешнее совместительство'].groupby('inn')[['fot', 'fot_aup', 'fot_isp']].sum().reset_index()), path_to_dir + 'stats по ' + months[-1])

In [92]:
static_names = {'inn': (np.nan, 'ИНН'), 'snils': (np.nan, 'СНИЛС'), 'szp': (np.nan, 'СЗП'), 'szp_teach': (np.nan, 'СЗП учителей'),
'szp_ped': (np.nan, 'СЗП педы'), 'fot': (np.nan, 'ФОТ'), 'fot_aup': (np.nan, 'ФОТ АУП'), 'top10_all': (np.nan, 'max 10% от всех работников'),
'bot10_all': (np.nan, 'min 10% от всех работников'), 'top10_teach': (np.nan, 'max 10% учителей'), 'bot10_teach': (np.nan, 'min 10% учителей'),
'bot50_teach': (np.nan, 'min 50% учителей'), 'bot50_ped': (np.nan, 'min 50% педагогов'), 'fot_isp': (np.nan, 'ФОТ НОУВП')}
per_month_names = {'sum': (np.nan, 'Начисления'), 'job': (np.nan, 'Должность'), 'day': (np.nan, 'Отработанный месяц'),
'type': (np.nan, 'Вид занятости'), 'stv': (np.nan, 'Ставка'), 'status': (np.nan, 'Статус на конец месяца'), 'status_pref': (np.nan, 'Статус на начало месяца'),
'wday': (np.nan, 'Отработано дней'), 'nday': (np.nan, 'Всего рабочих дней в месяце')}
monthly_names = {key: value for key, value in zip([key + '_' + month for key in per_month_names.keys() for month in months_year], [(month, value[1]) for value in per_month_names.values() for month in months_year_rus])}
q_names = {key: value for key, value in zip([key + '_q_' + str(q) for key in static_names.keys() for q in range(1, 5)], [('кв_' + str(q), value[1]) for value in static_names.values() for q in range(1, 5)])}
translate = static_names | per_month_names | monthly_names | q_names

In [97]:
def print_with_trans(df_in, file, trans):
    df = df_in.copy()
    df.columns = pd.MultiIndex.from_tuples(df.rename(columns=trans).columns.to_list())
    row = [[] for i in df.columns.names]
    wb = Workbook()
    ws = wb.active
    for i in df.columns:
        for j in range(len(row)):
            row[j].append(i[j])
    for i in row:
        ws.append(i)
    for i in df_to_row(df, header=False, index=False):
        ws.append(i)
    wb.save(file + '.xlsx')

In [94]:
print_with_trans(for_fot, path_to_dir + 'test_fot', translate)
print_with_trans(res, path_to_dir + 'test', translate)

In [95]:
print_df(res, path_to_dir + 'accum по ' + months[-1])

In [96]:
print_df(for_fot, path_to_dir + 'accum_fot по ' + months[-1])